# Conclude the Distribution Function of Electron Produced by the Annihilation of Dark Matter


## 1.Parameters
in this part, we have to define some parameters in our study, there are some rules:
 * Formula calculation uses the **natural unit** system 
 * Numerical calculation convert natural units to the International System of units, length units use **$pc$** and **$cm$**, time units use $s$ and $yr$, mass units use $M_\odot$ ,$g$ and $GeV$
 * So that we need unit conversion, like "A2B", means "the value of A in unit B"

In [2]:
import autograd.numpy as np
from autograd import grad
from scipy.integrate import quad
from scipy.interpolate import RegularGridInterpolator
from scipy.interpolate import interp1d
import os
from pathlib import Path
from scipy.integrate import solve_ivp
from tqdm import tqdm
import sympy as sp
import matplotlib.pyplot as plt
import pandas as pd
from typing import Callable, List
import multiprocessing
import concurrent.futures

class Parameters:
    def __init__(self, spin=0.9375):
        #Unit conversion
        self.GeV2cm=1.98e-14 #GeV -> cm
        self.GeV2s=6.58e-25 #GeV -> s
        self.g2GeV=5.62e23 #g -> GeV
        self.B2GeV=1.95e-20 #Gauss -> GeV
        self.pc2cm=3.086e18 # pc -> cm
        self.yr2s=3.15576e7 # yr -> s

        #Physical constants
        self.c=2.99792458e10 #speed of light, (cm/s)
        self.m_planck=1.221e19 #Planck mass=G^-1/2, (GeV)
        self.sigma_T=6.6524586e-25 #Thomson cross section, (cm^2)
        self.m_e=0.511e-3 #electron mass, (GeV)

        #Black Hole parameters
        self.aJ=spin #dimensionless angular momentum value of M87, (1)
        self.M_sun=1.989e33 #mass of the sun, (g)
        self.M_BH=6.5e9 #mass of the black hole, (M_sun)
        self.r_g=(self.M_BH*self.M_sun/self.m_planck**2)*self.g2GeV*self.GeV2cm/self.pc2cm #r_g=GM/c^2, (pc)
        self.r_gE=self.M_BH*self.M_sun*self.g2GeV/self.m_planck**2


## 2.Magnetic field and magnetic induction line
In cylindrical coordinates, $(x,\phi,z)=(r\sin \theta,\phi, r\cos\theta)$

Define dimensionless variables
$$\mathcal{R}\equiv\sqrt{\mathcal{X}^{2}+\mathcal{Z}^{2}}\equiv\frac{r}{r_{q}},\quad\mathcal{B}\equiv\frac{B}{1G}.$$
set magnetic field unit vector
$$
\frac{\bm{B}}{|B|}=b_r\hat{e}_r+b_\theta \hat{e}_\theta+b_\phi \hat{e}_{\phi}
$$


In [ ]:
'''
first, we have to define (r,theta) in jet region or disk region
'''
class RegionClassifier:
    def __init__(self, a_J=0.9375):
        self.a=a_J
        self.r_H=1+(1-self.a**2)**0.5 #Horizon radius, (r_g)
        self.x=[-0.0011664736219212544-0.000886998813904891j,-0.0011664736219212544+0.000886998813904891j,0.0008537370352110573-0.0012240788722115343j,0.0008537370352110573+0.0012240788722115343j]
        self.r_g=(6.5e9*1.989e33/1.221e19**2)*5.62e23*1.98e-14/3.086e18 #r_g=GM/c^2, (pc)


    def theta_jet(self,r):
        return np.arccos(1-self.r_H/r)
    
    def S(self,r):
        s=0
        for i in range(4):
            s+=(500-200*(self.x[i]/self.r_g)-21*(self.x[i]/self.r_g)**2)/(25+4*(self.x[i]/self.r_g)+3*(self.x[i]/self.r_g)**2+2*(self.x[i]/self.r_g)**3)*np.log(r-(self.x[i]/self.r_g))
        return s

    def theta_stag(self,r,theta,r_stag):
        z=0.5*np.arccos(-(2/r_stag**2)*(2*r_stag+0.5*(-4*r-r**2*np.cos(2*theta)+2*self.S(r))-self.S(r_stag)))
        return z.real

    def theta_disk(self,r):
        return self.theta_stag(0,1,r)

    def classify(self, r, theta):
        theta_j=self.theta_jet(r)
        theta_d=self.theta_disk(r)
        if theta < theta_j:
            return 'jet'
        elif theta > theta_d:
            return 'disk'
        else:
            return 'jet-disk'


### 2.1 jet region
A self-similarity variable $\Psi\equiv\mathcal{R}(1-\cos\theta)$, magnetic flux $\Phi(\Psi)$ and angular velocity of magnetic field lines $\Omega_F(\Psi)$. The magnetic field components can be depicted as:
$$
\begin{aligned}
 & \mathcal{B}_{x}=\mathcal{X}\Phi^{\prime}/[2\pi\mathcal{R}(\mathcal{Z}+\mathcal{R})], \\
 & \mathcal{B}_{\phi}=\Omega_Fg_{\phi\phi}\Phi^{\prime}/(2\pi\mathcal{R}), \\
 & \mathcal{B}_{z}=\Phi^{\prime}/(2\pi\mathcal{R}),
\end{aligned}
$$
with $\Phi^{\prime}\equiv\mathrm{d}\Phi/\mathrm{d}\Psi$, the ansatz of $\Phi^{\prime}$ is given by
$$
\Phi^{\prime}(\Psi)=c_1\exp(-c_2\cdot\Psi),
$$ 
$\Omega_F$ is given by
$$
\Omega_F(\Psi,a_J)=\frac{\sin^2\theta_H(1+\ln\mathcal{G})}{4\ln2+\sin^2\theta_H+(\sin^2\theta_H-2\mathcal{G})\ln\mathcal{G}}\Omega_H,
$$
where $\Omega_H=a_J/\mathcal{R}_H$, $\mathcal{R}_{H}=(1+\sqrt{1-a_{J}^{2}})$, $\mathcal{G}=1+\cos\theta_H$, $\Psi=\mathcal{R}_H(1-\cos\theta_H).$

also
$$
g_{\phi\phi}=(\mathcal{R}^2+a_J^2+\frac{2\mathcal{R}a_J^2}{\mathcal{R}^2+a_J^2\cos^2\theta}\sin^2\theta)\sin^2\theta
$$


### 2.2 disk region
$$
\frac{\mathcal{B}_x}{\mathcal{B}_z}=\frac{\mathcal{Z}\cdot K(\mathcal{R})}{\mathcal{X}},
$$
with $K(\mathcal{R})=1000\hat{\mathcal{R}^{-4}}+100\mathcal{R}^{-3}+8\mathcal{R}^{-2}+4\mathcal{R}^{-1}+1.$
The overall magnitude of the magnetic field and its $\phi$-component are determined by
$$
\mathcal{B}_{2D}=\sqrt{\mathcal{B}_x^2+\mathcal{B}_z^2}=b_1\mathcal{R}^{b_2\sin(4\theta-b_3)-b_4},\\\mathcal{B}_{\phi}=a_1\mathcal{R}^{a_2\sin(4\theta-a_3)-a_4},
$$

In [ ]:
class Physics:
    '''
    physics field, contains subclass such as MagneticField, VelocityField and NablaV. 
    '''
    def __init__(self, a_J=0.9375):
        self.params=Parameters()
        self.r_g=self.params.r_g
        self.classifier=RegionClassifier(a_J)
        self.aJ=a_J
        self.r_H=1+(1-self.aJ**2)**0.5
        self.Omega_H=self.aJ/self.r_H
        self.x=[-0.0011664736219212544-0.000886998813904891j,-0.0011664736219212544+0.000886998813904891j,0.0008537370352110573-0.0012240788722115343j,0.0008537370352110573+0.0012240788722115343j]

    def Phi(self,r,theta):
        return r*(1-np.cos(theta)) 
    
    def theta_H(self,Phi):
        return np.arccos(1-Phi/self.r_H)
    
    def gphiphi(self,r,theta):
        return (r**2+self.aJ**2+(2*r*self.aJ**2*np.sin(theta)**2)/(r**2+self.aJ**2*np.cos(theta)**2))*np.sin(theta)**2
    
    def Gg(self,Phi):
        return 1+np.cos(self.theta_H(Phi))
    
    def Omega_F(self,Phi):
        if Phi==0:
            return 0
        else:
            sin=np.sin(self.theta_H(Phi))
            lnGg=np.log(self.Gg(Phi))
            Gg=self.Gg(Phi)
            return (sin**2*(1+lnGg)*self.Omega_H)/(4*np.log(2)+sin**2+(sin**2-Gg)*lnGg)
        
    def S(self,r):
        s=0
        for i in range(4):
            s+=(500-200*(self.x[i]/self.r_g)-21*(self.x[i]/self.r_g)**2)/(25+4*(self.x[i]/self.r_g)+3*(self.x[i]/self.r_g)**2+2*(self.x[i]/self.r_g)**3)*np.log(r-(self.x[i]/self.r_g))
        return s

    def theta_stag(self,r,theta,r_stag):
        z=0.5*np.arccos(-(2/r_stag**2)*(2*r_stag+0.5*(-4*r-r**2*np.cos(2*theta)+2*self.S(r))-self.S(r_stag)))
        return z.real
    
    def r_stag_jet(self,Phi):
        if Phi==0:
            return 100
        else:
            return 1/(self.Omega_F(Phi)*np.sqrt(Phi))
    
    def theta_stag_jet(self,Phi):
        return np.arccos(1-Phi/self.r_stag_jet(Phi))
    
    def theta_inj_jet(self,r,Phi):
        return np.arccos(1-Phi/r)

In [ ]:
class MagneticField(Physics):
    def __init__(self,name='jet',a_J=0.9375):
        super().__init__(a_J)
        self.aJ=a_J
        self.name=name
        if self.aJ==0.9375:
            self.c=[123,0.679]
        elif self.aJ==0.5:
            self.c=[126,0.345]
        if self.aJ==0.9375:
            self.a=[38.1,0.511,2.34,1.81]
        elif self.aJ==0.5:
            self.a=[19.4,0.395,2.72,1.56]
        if self.aJ==0.9375:
            self.b=[52.1,0.579,2.39,2.29]
        elif self.aJ==0.5:
            self.b=[94.1,0.596,2.70,2.39]
    
    def dPhi(self,Phi):
        return self.c[0]*np.exp(-self.c[1]*Phi)
    
    def B_jet(self,r,theta):
        Phi=self.Phi(r,theta)
        dPhi=self.dPhi(Phi)
        Omega_F=self.Omega_F(Phi)
        gphiphi=self.gphiphi(r,theta)
        B_x=(np.sin(theta)*dPhi)/(2*np.pi*r*(1+np.cos(theta)))
        B_z=dPhi/(2*np.pi*r)
        B_phi=(Omega_F*gphiphi*dPhi)/(2*np.pi*r)
        B=np.sqrt(B_x**2+B_z**2+B_phi**2)
        b_r=(B_x*np.sin(theta)+B_z*np.cos(theta))/B
        b_theta=(-B_z*np.sin(theta)+B_x*np.cos(theta))/B
        return B, b_r, b_theta
    
    def K(self,r):
        return 1000*r**(-4)+100*r**(-3)+8*r**(-2)+4*r**(-1)+1
    
    def B_disk(self,r,theta):
        B_2D=self.b[0]*r**(self.b[1]*np.sin(4*theta-self.b[2])-self.b[3])
        B_phi=self.a[0]*r**(self.a[1]*np.sin(4*theta-self.a[2])-self.a[3])
        B=np.sqrt(B_2D**2+B_phi**2)
        k=self.K(r)
        b_r=(B_2D/B)*((1+k)*np.sin(theta))/np.sqrt(k**2+np.tan(theta)**2)
        if theta==np.pi/2:
            b_theta=-B_2D/B
        else:
            b_theta=-B_2D/B*(np.sin(theta)**2*(1-k/(np.tan(theta)**2)))/(np.cos(theta)*np.sqrt(k**2+np.tan(theta)**2))
        return B, b_r, b_theta

    def B(self,r,theta):
        if self.name=='jet':
            return self.B_jet(r,theta)
        elif self.name=='disk':
            return self.B_disk(r,theta)

## 3. Bulk Velocity of Electron-Positron Flows
### 3.1 jet region
the effective potential in jet region
$$
V_{eff}=-\frac{r_g}{r}-\frac{1}{2}\Omega^2r^2\sin^2\theta
\\V_{eff}=-\frac{r_g}{r}-\frac{\Omega^2}{2 r_g^2}(2\Psi r-\Psi^2)
$$
the bulk velocity magnitude
$$
v_b=\sqrt{1-e^{2[V_{eff}(r,\theta)-V_{eff}(r_{stag},\theta_{stag})]}}
$$
where $(r_{stag}, \theta_{stag})$ denotes the intersection of the magnetic field line, traversing through $(r, \theta)$, with the stagnation surface.
thus, 
$$
r_{stag}(\Psi)=\begin{cases}
100r_g, \ \ \ \ \ \ \ \ \Psi=0 \\
\frac{r_g}{\Omega_F(\Psi)\sqrt{\Psi}}\ \ \ \ \ \ \Psi\ne 0
\end{cases}
$$
thus,
$$
\theta_{stag}(\Psi,r_{stag})=\arccos (1-\frac{\Psi}{r_{stag}}) 
$$

### 3.2 disk region
the radial infall velocity 
$$
v_b=\sqrt{\frac{2r_g}{r}}
$$
for $(r_{stag}, \theta_{stag})$
$$
\theta_{stag}(r,\theta, r_{stag})=Re[\frac{1}{2}\arccos [-\frac{2}{r_{stag}^2}(2r_gr_{stag}+\frac{1}{2}(-4rr_g-r^2\cos(2\theta)+2r_g\times S(r))-r_g\times S(r_{stag}))]]
$$
with
$$
S(u)=\sum_{i=1}^4\frac{500R_g^4\ln(u-x_i)-200R_g^3x_i\ln(u-x_i)-21R_g^2x_i^2\ln(u-x_i)}{25R_g^3+4R_g^2x_i+3R_gx_i^2+2x_i^3}.
$$
and, Boundary of the disk region in the x-z plane, lower bound of $\Theta$ in the disk region, defined by the boundary magnetic field lines resembling hyperbolas in the x-z plane .
$$
\theta_{bound}=\theta_{stag}(0,1,r_{stag})
$$



In [ ]:
class Velocity(Physics):
    def __init__(self,name='jet',a_J=0.9375):
        super().__init__(a_J)
        self.name=name

    def V_eff(self,r,Phi):
        return -1/r-self.Omega_F(Phi)**2*(2*Phi*r-Phi**2)/2

    def v_jet(self,r,theta):
        Phi=r*(1-np.cos(theta))
        if theta==0:
            vb=np.sqrt(1-np.exp(2*self.V_eff(r,Phi)))
        else:
            r_stag=self.r_stag_jet(Phi)
            theta_stag=self.theta_stag_jet(Phi)
            Phi_stag=r_stag*(1-np.cos(theta_stag))
            vb=np.sqrt(1-np.exp(2*(self.V_eff(r,Phi)-self.V_eff(r_stag,Phi_stag))))
        if vb<=0.001:
            vb=0.001
        return vb

    def v_disk(self,r,theta):
        return np.sqrt(2/r)

    def v(self,r,theta):
        if self.name=='jet':
            return self.v_jet(r,theta)
        elif self.name=='disk':
            return self.v_disk(r,theta)


## 4. Radiation loss and adiabatic compression
### 4.1 synchrotron radiation
$$
\dot{p}_{syn}=-\frac{\sigma_T}{6\pi m_e^2}p^2B^2
$$
### 4.2 Adiabatic Compression
$$
\dot{p}_{adi}=-\frac{p}{3}(\nabla\cdot\bm{v})
$$
#### 4.2.1 jet region
$$
\nabla\cdot\bm{v}=\begin{cases}
 \frac{1}{r^{2}}\frac{\partial}{\partial r}(r^{2}v_{0}^{\mathrm{jet}})-\frac{v_0^{\mathrm{jet}}}{r}\ \ \ \ \ \ \ \ \ \ \theta=0\\
\frac{1}{r^{2}}\frac{\partial}{\partial r}(r^{2}v_{r}^{\mathrm{jet}})+\frac{1}{r\sin\theta}\frac{\partial}{\partial\theta}(\sin\theta v_{\theta}^{\mathrm{jet}})\ \ \ \theta\ne 0
\end{cases}
$$
#### 4.2.2 disk region
$$
\nabla\cdot\mathbf{v}_{\mathrm{disk}}=\sqrt{2}r_g^{1/2}\left\{\frac{1}{r^2}\frac{\partial}{\partial r}(r^{3/2}b_r)+\frac{1}{r\sin\theta}\frac{\partial}{\partial\theta}(\sin\theta r^{-1/2}b_\theta)\right\}.
$$

In [ ]:
# in this class, we don't use dimensionless physical quantities
class PDerivative(MagneticField,Velocity):
    def __init__(self,name='jet',direction='in',a_J=0.9375):
        super().__init__(name,a_J)
        self.name=name
        self.dir=direction
        self.params=Parameters()
        self.k_theta=(self.params.sigma_T*(1/self.params.GeV2cm**2)*self.params.B2GeV**2)/(6*np.pi*self.params.m_e**2*self.params.r_g)*self.params.r_gE
        self.dr2vr_dr=grad(self.r2vr,0)
        self.dr2sin_dtheta=grad(self.r2sin,0)
        self.dr32br_dr=grad(self.r32br,0)
        self.dsinr12btheta_dtheta=grad(self.sinr12btheta,0)

    def dP_syn(self,B):
        return self.k_theta*self.params.r_g*B**2
    # length unit is in rg

    def r2vr(self,r,theta):
        if theta==0:
            return r**2*self.v_jet(r,theta)
        else:
            return r**2*self.v_jet(r,theta)*self.B_jet(r,theta)[1]
    
    def r2sin(self,theta,r):
        return np.sin(theta)*self.v_jet(r,theta)*self.B_jet(r,theta)[2]
    
    def r32br(self,r,theta):
        return r**1.5*self.B_disk(r,theta)[1]
    
    def sinr12btheta(self,theta,r):
        return np.sin(theta)*r**(-0.5)*self.B_disk(r,theta)[2]

    def nablaV_jet(self,r,theta):
        if theta==0:
            return (1/r**2)*self.dr2vr_dr(r,theta)-self.v_jet(r,theta)/r
        else:
            return (1/r**2)*self.dr2vr_dr(r,theta)+(1/r*np.sin(theta))*self.dr2sin_dtheta(theta,r)

    def nablaV_disk(self,r,theta):
        return np.sqrt(2)*((1/r**2)*self.dr32br_dr(r,theta)+(1/(r*np.sin(theta)))*self.dsinr12btheta_dtheta(theta,r))

    def dP_jet_in(self,r,theta,p):
        B=self.B_jet(r,theta)
        return (-self.dP_syn(B[0])+(p/3)*(self.nablaV_jet(r,theta)))/(self.v_jet(r,theta)*B[1])

    def dP_jet_out(self,r,theta,p):
        B=self.B_jet(r,theta)
        return (self.dP_syn(B[0])+(p/3)*(self.nablaV_jet(r,theta)))/(self.v_jet(r,theta)*B[1])

    def dP_disk(self,r,theta,p):
        B=self.B_disk(r,theta)
        return (-self.dP_syn(B[0])+(p/3)*(self.nablaV_disk(r,theta)))/(self.v_disk(r,theta)*B[1])

    def dP(self,r,theta,p):
        if self.name=='jet':
            if self.dir=='in':
                return self.dP_jet_in(r,theta,p)
            elif self.dir=='out':
                return self.dP_jet_out(r,theta,p)
        elif self.name=='disk':
            return self.dP_disk(r,theta,p)
    


## 5. Numerical Integration prepare
In the next section, we will deal with numerical calculations and store some intermediate variables. 

### 5.1 Integration cutoff point
First, we have to confirm the Integral upper bound along the line. We set a general limit
$$
r_{boundary}=30 r_g
$$
To prevent possible divergence of $p_{inj}=p_{inj}(p,r,r_{inj})$ when computing the change in electron momentum during transport, the divergence point of $p_{inj}$ is pre computed.

we use numerical solution of differential equations 
$$
\frac{dp'}{dr}=\frac{\dot{p}'_{adi}+\dot{p}'_{syn}}{v_r}=\frac{-\frac{1}{3}p'\nabla\cdot\bm{v}+k_{\theta}B^2}{v_r}
$$
with $p'=1/p$, and stop at large $p=10^6 GeV(p'=1/p=10^{-6}GeV^{-1})$.

If no divergence occurs,the computation will proceed up to the boundary of the domain.


In [ ]:
def cutoffPoint():
    phy=Physics()
    pDe_jet_in=PDerivative('jet','in')
    pDe_jet_out=PDerivative('jet','out')
    pDe_disk=PDerivative('disk')
    classify=RegionClassifier()

    # grid parameters
    R_min=2.5 
    R_max=2.5*12
    R_bound=30
    mm=1 # for theta
    nn=1 # for r
    p_inj_down=np.float64(1e-6)

    logr_grid = np.linspace(np.log10(R_min), np.log10(R_max), nn + 1)
    r_grid = np.exp(logr_grid*np.log(10))
    theta_grid = np.linspace(0, 0.5*np.pi - 1e-3, mm + 1)
    p_list=np.logspace(-3,4,2)

    def cutoffPoint(r_grid,theta_grid):
        results=np.zeros((nn+1,mm+1,len(p_list),4),dtype=float)
        for i, r in enumerate(r_grid):
            for j, theta in enumerate(theta_grid):
                r_start=r
                r_end=R_bound
                # judge the region
                region=classify.classify(r,theta)
                if region=='jet':
                    Phi=phy.Phi(r,theta)
                    r_stag=phy.r_stag_jet(Phi)
                    if r<r_stag:
                        r_end=min(R_bound,r_stag)
                        ode_fun = lambda R, P: pDe_jet_in.dP(R,phy.theta_inj_jet(R,Phi), P)
                    else:
                        r_end=r_stag
                        ode_fun = lambda R, P: pDe_jet_out.dP(R,phy.theta_inj_jet(R,Phi), P)
                elif region=='disk':
                    ode_fun = lambda R, P: pDe_disk.dP(R,phy.theta_stag(r,theta,R), P)

                for k, p in enumerate(p_list):
                    results[i,j,k,0]=r
                    results[i,j,k,1]=theta
                    results[i,j,k,2]=p
                    if region=='jet-disk':
                        results[i,j,k,3]=0.0
                    else:
                        def hit_aim(R,P): return P-p_inj_down
                        hit_aim.terminal = True
                        hit_aim.direction = -1

                        direction=np.sign(r_end-r_start)
                        if direction==0:
                            results[i,j,k,3]=r_end
                            continue
                        sol=solve_ivp(
                            fun=ode_fun,
                            t_span=(r_start, r_end),
                            y0=[1/p],
                            events=hit_aim,
                            max_step=abs(r_end-r_start)/100,
                        )
                        evts=sol.t_events[0]
                        if len(evts)>0:
                            results[i,j,k,3]=sol.t_events[0][0]
                        else:
                            results[i,j,k,3]=r_end
        return results
    
    results=cutoffPoint(r_grid,theta_grid)
    return results

cutoffresults=cutoffPoint()
np.save('cutoffPoint.npy',cutoffresults)

In [ ]:
datacutoff=np.load("cutoffPoint.npy")
print(datacutoff.shape)
for i in range(datacutoff.shape[0]):
    for j in range(datacutoff.shape[1]):
        for k in range(datacutoff.shape[2]):
            print(datacutoff[i,j,k,0],datacutoff[i,j,k,1],datacutoff[i,j,k,2],datacutoff[i,j,k,3])

In [ ]:
datacutoff=np.load('cutoffPoint.npy')
i=1
for j in range(datacutoff.shape[1]):
    for k in range(datacutoff.shape[2]):
        print(datacutoff[i,j,k,0],datacutoff[i,j,k,1],datacutoff[i,j,k,2],datacutoff[i,j,k,3])

### 5.2 Calculate intermediate function $G(r,\theta)$
the definition of $G(r,\theta)$
$$
G(s_{\vec{r}}) \propto \exp\left[ -\int_{s_{0}}^{s_{\vec{r}}} \frac{4}{3} \frac{\nabla \cdot \vec{v}_{b}(s)}{v_{b}(s)} \, \mathrm{d}s \right] = \exp\left[ -\int_{s_{0}}^{s_{\vec{r}}} \frac{4}{3} \left( \nabla \cdot \hat{v}_{b}(s) + \frac{1}{v_{b}(s)} \frac{\mathrm{d}v_{b}}{\mathrm{d}s}(s) \right) \mathrm{d}s \right].
$$


In [ ]:
def G_jet():
    phy=Physics()
    pDe_jet_in=PDerivative('jet','in')
    vel_jet=Velocity('jet')
    mag=MagneticField('jet')
    
    ll=2
    mm=1
    r_bound=30
    jet_results=[]
    for i in range(1,ll+2):
        logr=np.log10(2.5)+(i-1)*np.log10(r_bound/2.5)/ll
        r=np.exp(logr*np.log(10))
        psis=np.linspace(0,phy.r_H,mm+1)

        for psi in psis:
            def integrand(x):
                theta=phy.theta_inj_jet(np.exp(x),psi)
                B=mag.B_jet(np.exp(x),theta)
                return np.exp(x)*(4.0/3.0)*(pDe_jet_in.nablaV_jet(np.exp(x),theta)/(vel_jet.v_jet(np.exp(x),theta)*B[1]))

            x_min=np.log(10)*logr
            x_max=np.log(2.5)
            integral_value, _ =quad(integrand, x_min, x_max)
            G_val=np.exp(integral_value)
            jet_results.append([r,psi,G_val])
    
    jet_results=np.array(jet_results)
    return jet_results

Gjetresult=G_jet()
np.save('G_jet.npy',Gjetresult)

In [ ]:
dataGjet=np.load("G_jet.npy")
print(dataGjet.shape)
for i in range(dataGjet.shape[0]):
    print("r: ",dataGjet[i,0],"psi: ",dataGjet[i,1],"G: ",dataGjet[i,2])

In [ ]:
def G_disk():
    phy=Physics()
    pDe_disk=PDerivative('disk')
    vel_disk=Velocity('disk')
    mag=MagneticField('disk')
    
    ll=2
    mm=1
    r_bound=30
    disk_results=[]

    for i in range(1,ll+2):
        logr=np.log10(2.5)+(i-1)*np.log10(r_bound/2.5)/ll
        r=np.exp(logr*np.log(10))
        thetas=np.linspace(0.25*np.pi,0.5*np.pi-0.001,mm+1)

        for theta in thetas:
            def integrand(x):
                theta_inj=phy.theta_stag(r,theta,x)
                B=mag.B_disk(x,theta_inj)
                return (4.0/3.0)*(pDe_disk.nablaV_disk(x,theta_inj)/(vel_disk.v_disk(x,theta_inj)*B[1]))

            integral_value, _ =quad(integrand, r, r_bound)
            G_val=np.exp(integral_value)
            disk_results.append([r,theta,G_val])
    
    disk_results=np.array(disk_results)
    return disk_results

Gdiskresult=G_disk()
np.save('G_disk.npy',Gdiskresult)

In [ ]:
dataGdisk=np.load("G_disk.npy")
print(dataGdisk.shape)
for i in range(dataGdisk.shape[0]):
    print("r: ",dataGdisk[i,0],"r0: ",dataGdisk[i,1],"G: ",dataGdisk[i,2])

### 5.3 Calculate intermediate function $\xi(r,\theta)$
the definition of $\xi$
$$
\xi(r,\theta)=G^{1/4}(r,\theta)
$$

### 5.4 Calculate intermediate function $\beta(r,\theta)$
the definition of $\beta$
$$
\begin{align*}
\beta(s_{\rm inj}, s_{\vec{r}}) \equiv \frac{e^4}{36\pi^2 m_e^4} \int_{s_{\vec{r}}}^{s_{\rm inj}} \frac{B(s')^2}{v_b(s')} \exp\left[ -\int_{s_{\vec{r}}}^{s'} \frac{\nabla \cdot \vec{v}_b(s'')}{3\,v_b(s'')} \, \mathrm{d}s'' \right] \mathrm{d}s',
\end{align*}
$$
thus
$$
\beta(r_{inj},\theta_{inj},r,\theta)=\int_{r}^{r_{inj}}\frac{\dot{p}'_{syn}(x,\theta_{x})}{v_r(x,\theta_{x})}\frac{\xi(x,\theta_x)}{\xi(r,\theta)}dx
$$

In [ ]:
from scipy.interpolate import RegularGridInterpolator

def beta_jet():
    phy=Physics()
    pDe_jet_in=PDerivative('jet','in')
    vel_jet=Velocity('jet')
    mag=MagneticField('jet')

    ll=2
    mm=1
    r_bound=30
    jet_results=[]

    for i in range(1,ll+2):
        logr=np.log10(2.5)+(i-1)*np.log10(r_bound/2.5)/ll
        r=np.exp(logr*np.log(10))
        psis=np.linspace(0,phy.r_H,mm+1)

        for psi in psis:
            theta=phy.theta_inj_jet(r,psi)
            def integrand(x):
                thetax=phy.theta_inj_jet(np.exp(x),psi)
                B=mag.B_jet(np.exp(x),thetax)
                return np.exp(x)*(pDe_jet_in.dP_syn(B[0])/(vel_jet.v_jet(np.exp(x),thetax)*B[1]))*(xi_jet_interp(np.exp(x),thetax)/xi_jet_interp(r,theta))

            x_min=np.log(10)*logr
            x_max=np.log(2.5)
            integral_value, _ =quad(integrand, x_min, x_max)
            beta_val=integral_value
            jet_results.append([r,psi,beta_val])
    
    jet_results=np.array(jet_results)
    return jet_results

def xi_interpolator_jet():
    jet_data=np.load("G_jet.npy")
    rs,psis=np.unique(jet_data[:,0]),np.unique(jet_data[:,1])
    Gs=jet_data[:,2].reshape(len(rs),len(psis))

    xis=Gs**0.25
    xijet_interp=RegularGridInterpolator((rs,psis),xis,bounds_error=False,fill_value=None)
    def xi_jet(r,theta):
        psi=r*(1-np.cos(theta))
        return xijet_interp((r,psi))
    
    return xi_jet
    
def beta_interpolator_jet():    
    beta_jet_data=np.load("beta_jet.npy")
    rss,psiss=np.unique(beta_jet_data[:,0]),np.unique(beta_jet_data[:,1])
    betas=beta_jet_data[:,2].reshape(len(rss),len(psiss))
    betajet_interp=RegularGridInterpolator((rss,psiss),betas,bounds_error=False,fill_value=None)

    def beta_jet(r,theta):
        psi=r*(1-np.cos(theta))
        return betajet_interp((r,psi))

    return beta_jet

xi_jet_interp=xi_interpolator_jet()
betajetresult=beta_jet()
np.save('beta_jet.npy',betajetresult)
beta_jet_interp=beta_interpolator_jet()

In [ ]:
from scipy.interpolate import RegularGridInterpolator

def beta_disk():
    phy=Physics()
    pDe_disk=PDerivative('disk','in')
    vel_disk=Velocity('disk')
    mag=MagneticField('disk')

    ll=2
    mm=1
    r_bound=30
    disk_results=[]

    for i in range(1,ll+2):
        logr=np.log10(2.5)+(i-1)*np.log10(r_bound/2.5)/ll
        r=np.exp(logr*np.log(10))
        thetas=np.linspace(0.25*np.pi,0.5*np.pi-0.001,mm+1)

        for theta in thetas:
            def integrand(x):
                thetax=phy.theta_stag(r,theta,x)
                B=mag.B_disk(x,thetax)
                return (pDe_disk.dP_syn(B[0])/(vel_disk.v_disk(x,thetax)*B[1]))*(xi_disk_interp(x,thetax)/xi_disk_interp(r,theta))

            integral_value, _ =quad(integrand, r, r_bound)
            beta_val=integral_value
            disk_results.append([r,theta,beta_val])
    
    disk_results=np.array(disk_results)
    return disk_results

def xi_interpolator_disk():
    disk_data=np.load("G_disk.npy")
    rs,thetas=np.unique(disk_data[:,0]),np.unique(disk_data[:,1])
    Gs=disk_data[:,2].reshape(len(rs),len(thetas))

    xis=Gs**0.25
    xijet_interp=RegularGridInterpolator((rs,thetas),xis,bounds_error=False,fill_value=None)
    def xi_disk(r,theta):
        return xijet_interp((r,theta))
    
    return xi_disk
    
def beta_interpolator_disk():    
    beta_disk_data=np.load("beta_disk.npy")
    rss,thetass=np.unique(beta_disk_data[:,0]),np.unique(beta_disk_data[:,1])
    betas=beta_disk_data[:,2].reshape(len(rss),len(thetass))
    betajet_interp=RegularGridInterpolator((rss,thetass),betas,bounds_error=False,fill_value=None)

    def beta_jet(r,theta):
        return betajet_interp((r,theta))

    return beta_jet

xi_disk_interp=xi_interpolator_disk()
betadiskresult=beta_disk()
np.save('beta_disk.npy',betadiskresult)
beta_disk_interp=beta_interpolator_disk()


In [ ]:
databetajet=np.load("beta_jet.npy")
print(databetajet.shape)
for i in range(databetajet.shape[0]):
    print("r: ",databetajet[i,0],"psi: ",databetajet[i,1],"beta: ",databetajet[i,2])

In [ ]:
betadiskresult=np.load("beta_disk.npy")
print(betadiskresult.shape)
for i in range(betadiskresult.shape[0]):
    print("r: ",betadiskresult[i,0],"r0: ",betadiskresult[i,1],"beta: ",betadiskresult[i,2])

### 5.5 Calculate inject momentum $p_{inj}$
Analytic calculation
$$
p_{inj}(r_{inj},\theta_{inj})=\{\frac{\xi(r,\theta)}{\xi(r_{inj},\theta_{inj})}[\frac{1}{p}-\beta(r_{inj},\theta_{inj},r,\theta)]\}^{-1}
$$
$$
\beta(r_{inj},\theta_{inj},r,\theta)=\beta(r,\theta)-\beta(r_{inj},\theta_{inj})
$$

In [ ]:
def save_file(path, ii, jj, data):
    outfile = Path(path)/"p_inj" /f"r_{ii}"/f"theta_{jj}.npy"
    os.makedirs(outfile.parent, exist_ok=True)
    np.save(outfile, data)

def process_pair(args):
    ii, jj, r_vals, theta_vals, cutoff_data, ll, path,classify, phy = args
    r_bin = r_vals[ii-1]
    theta_bin = theta_vals[jj-1]

    sub=cutoff_data[ii-1,jj-1,:,:]
    if sub.size == 0:
        return

    p0_vals = sub[...,2]
    rcut_vals = sub[...,3]
    n_pts=len(p0_vals)
    region = classify.classify(r_bin, theta_bin)

    if region == 'jet-disk':
        out = np.zeros((n_pts*ll, 3), dtype=float)
        out[:,0] = np.repeat(p0_vals, ll)
        save_file(path, ii, jj, out)
        return

    if region == 'jet':
        phi = phy.Phi(r_bin, theta_bin)
        R_stag = phy.r_stag_jet(phi)
        if r_bin > R_stag:
            first = np.linspace(0, 5/(ll-26), 30, endpoint=False)
            second = np.linspace(5/(ll-26), 1, ll-30)
            Rinjtab = np.concatenate([first, second])
            rtab = np.vstack([
                rcut_vals[i]+(r_bin-rcut_vals[i])*Rinjtab for i in range(n_pts)
            ])
            rtab_flat = rtab.ravel()
            theta_tab = np.array([
                phy.theta_inj_jet(rtab_flat[k], phi) for k in range(len(rtab_flat))
            ])
        else:
            first = np.linspace(0, 1, ll-30, endpoint=False)
            tail_start = (ll-31)/(ll-26)
            tail = np.linspace(tail_start, tail_start + 5/(ll-26), 30)
            Rinjtab = np.concatenate([first, tail])
            rtab = np.vstack([
                r_bin+(rcut_vals[i]-r_bin)*Rinjtab for i in range(n_pts)
            ])
            rtab_flat = rtab.ravel()
            theta_tab = np.array([
                phy.theta_inj_jet(rtab_flat[k], phi) for k in range(len(rtab_flat))
            ])
    else:
        first = np.linspace(0, 1, ll-30, endpoint=False)
        tail_start = (ll-31)/(ll-26)
        tail = np.linspace(tail_start, tail_start + 5/(ll-26), 30)
        Rinjtab = np.concatenate([first, tail])
        rtab = np.vstack([
            r_bin+(rcut_vals[i]-r_bin)*Rinjtab for i in range(n_pts)
        ])
        rtab_flat = rtab.ravel()
        theta_tab = np.array([
            phy.theta_stag(r_bin, theta_bin, rtab_flat[k]) for k in range(len(rtab_flat))
        ])

    pinj = np.empty((n_pts*ll, 3))
    for i in range(n_pts):
        for j in range(ll):
            idx = i * ll + j
            p0 = p0_vals[i]
            r_cur = rtab_flat[idx]
            theta_cur = theta_tab[idx]

            if region == 'jet':
                phi = phy.Phi(r_bin, theta_bin)
                R_stag = phy.r_stag_jet(phi)
                if r_bin > R_stag:
                    val = ((1/p0 + beta_jet_interp(r_bin, theta_bin)) *
                           (xi_jet_interp(r_bin, theta_bin) / xi_jet_interp(r_cur, theta_cur))
                           - beta_jet_interp(r_cur, theta_cur)) ** (-1)
                else:
                    val = ((1/p0 - beta_jet_interp(r_bin, theta_bin)) *
                           (xi_jet_interp(r_bin, theta_bin) / xi_jet_interp(r_cur, theta_cur))
                           + beta_jet_interp(r_cur, theta_cur)) ** (-1)
            else:
                val = ((1/p0 - beta_disk_interp(r_bin, theta_bin)) *
                       (xi_disk_interp(r_bin, theta_bin) / xi_disk_interp(r_cur, theta_cur))
                       + beta_disk_interp(r_cur, theta_cur)) ** (-1)
            pinj[idx] = (p0, r_cur, val)

    save_file(path, ii, jj, pinj)

def calculatePinj():
    classify = RegionClassifier()
    phy = Physics()

    path = Path.cwd()
    ll = 300
    cutoff_data = np.load(Path(path)/"cutoffPoint.npy")
    r_vals = np.unique(cutoff_data[...,0])
    theta_vals = np.unique(cutoff_data[...,1])
    nn = len(r_vals)-1
    mm = len(theta_vals)-1

    args_list = [
        (ii, jj, r_vals, theta_vals, cutoff_data, ll, path, classify, phy)
        for ii in range(1, nn+2)
        for jj in range(1, mm+2)
    ]

    with concurrent.futures.ProcessPoolExecutor() as executor:
        futures = [executor.submit(process_pair, args) for args in args_list]
        for _ in tqdm(concurrent.futures.as_completed(futures),
                      total=len(futures),
                      desc="Computing p_inj",
                      unit="task"):
            pass

if __name__ == '__main__':
    calculatePinj()


In [ ]:
datapinj=np.load("p_inj/r_2/theta_5.npy")
print(datapinj.shape)
for i in range(datapinj.shape[0]):
    print(datapinj[i,0],datapinj[i,1],datapinj[i,2])

## 6. Calculate distribution function $f_e$


Analytic calculation
$$
f_e(r,\theta,p)=\int_{s_{\vec{r}}}^{s_b}\mathrm{d}s_i\frac{q\left(s_i,p_i(s_i)\right)}{v_b(s_i)}\left(\frac{p_i(s_i)}{p}\right)^4G(s_{\vec{r}})G^{-1}(s_i).
$$
and electron generation rate
$$
q(\vec{r},p)=\frac{1}{4\pi p^2}\frac{\langle\sigma v\rangle\rho^2(\vec{r})}{2m_{\mathrm{DM}}^2}\sum_{i}\mathrm{BR}_{i}\frac{\mathrm{d}N_{e^{\pm},i}^{\mathrm{inj}}}{\mathrm{d}p}(p).
$$
the dark matter density
$$
\rho(\vec{r})=\left\{
\begin{array}
{ll}0 & \quad r<r_{\mathrm{crit}}(\theta,a_J), \\
\rho_{\mathrm{sat}}\equiv m_{\mathrm{DM}}/(\langle\sigma v\rangle t_{\mathrm{BH}}) & \quad r_{\mathrm{crit}}(\theta,a_J)\leq r<r_{\mathrm{sat}}, \\
\rho_{\mathrm{sp}}(r)\equiv\rho_{0}\left(r/r_{0}\right)^{-\gamma_{\mathrm{sp}}} & \quad r_{\mathrm{sat}}\leq r<r_{\mathrm{sp}}, \\
\rho_{\mathrm{halo}}(r)=\rho_{0}(r_{\mathrm{sp}}/r_{0})^{-\gamma_{\mathrm{sp}}}\left(r/r_{\mathrm{sp}}\right)^{-1} & \quad r\geq r_{\mathrm{sp}}.
\end{array}\right.
$$

### 6.1 dark matter density


In [ ]:
class BH_halo():
    def __init__(self,a_J=0.9375,m_DM=10,sigv=1e-28):
        self.aJ=a_J
        self.m_DM=m_DM
        self.sigv=sigv
        self.parame=Parameters()
        self.rho0=3.3e4
        self.r0=1e5*self.parame.r_g # Sun distance to GC, (pc)
        self.t_BH=1e8 # BH age, (year)
        self.rho_sat=self.m_DM/(self.sigv*self.t_BH*self.parame.yr2s) 
        self.gamma=1
        self.gamma_sp=(9-2*self.gamma)/(4-self.gamma)
        self.r_sat=self.r0*(self.rho0/self.rho_sat)**(1/self.gamma_sp)
        self.r_sp=0.001*(self.parame.M_BH*self.parame.M_sun*self.parame.g2GeV)**(1.5)*(self.rho0)**(-1.5)*(self.r0)**(-3.5)
        self._rmb_interpolator=self._prepare_rmb_grid()

    def rho_sp(self,r):
        return self.rho0*(r/self.r0)**(-self.gamma_sp)
    
    def rho_halo(self,r):
        return self.rho0*(self.r_sp/self.r0)**(-self.gamma_sp)*(r/self.r_sp)**(-1)
    
    def _prepare_rmb_grid(self):
        theta_vals=np.linspace(0, np.pi, 41)
        rmb_vals=[]
        r=sp.Symbol('r',real=True,positive=True)

        for th in theta_vals:
            theta = float(th)
            cos2 = np.cos(theta)**2
            sin = np.sqrt(1 - cos2)
            a = self.aJ

            expr = (r**4 - 4*r**3 - a**2*(1 - 3*cos2)*r**2 + a**4*cos2 +
                    4*a*sin * sp.sqrt(r**5 - a**2 * r**3 * cos2))
            
            try:
                roots=sp.nroots(expr,n=15,maxsteps=100,cleanup=True)
                roots_real=[float(rt.evalf()) for rt in roots if rt.is_real and rt>0]
                if roots_real:
                    rmb_vals.append(max(roots_real))
                else:
                    rmb_vals.append(1.0)
            except:
                rmb_vals.append(1.0)
        
        _rmb_interpolator = RegularGridInterpolator(
            (theta_vals,), np.array(rmb_vals), bounds_error=False, fill_value=None
        )
        return _rmb_interpolator

    def r_crit(self,theta):
        return float(self._rmb_interpolator([theta])[0])*self.parame.r_g
    
    def rho(self,r,theta):
        r_crit=self.r_crit(theta)
        if r<r_crit:
            return 0.0
        elif r<self.r_sat:
            return self.rho_sat
        elif r<self.r_sp:
            return self.rho_sp(r)
        else:
            return self.rho_halo(r)
    
    def plot_equatorial(self, num=200):
        rs = np.logspace(np.log10(0.5*self.r_crit(0)), np.log10(1e6*self.parame.r_g), num)
        rrs=rs/self.parame.r_g
        densities = [self.rho(rval, 0) for rval in rs]
        plt.loglog(rrs, densities)
        plt.xlabel(r'$r\ (\mathrm{Rg})$')
        plt.ylabel(r'$\rho(r,\theta=0)\ (\mathrm{GeV/cm}^3)$')
        plt.title('Dark Matter Spike Density Profile')
        plt.grid(True, which='both', ls='--')
        plt.show()
        
if __name__ == "__main__":
    model = BH_halo(a_J=0.9375,m_DM=10,sigv=1e-28)
    print(model.r_sat/model.parame.r_g)
    print(model.rho_sat)
    model.plot_equatorial()
        

### 6.2 the electron generation rate

In [ ]:
cwd_str = str(Path.cwd())  
mDMfilename = os.path.join(
    cwd_str,
    'pppc',
    'AtProduction_positrons',
    'AtProduction_positrons.dat'
)
mDMdata=np.loadtxt(mDMfilename,skiprows=1)
mDMdata=mDMdata[np.isclose(mDMdata[:,0],5)]
print(mDMdata)
data=[]
with open(mDMfilename, 'r') as f:
    for line in f:
        data.append(line.split())
        break
print(data[0])

In [ ]:
class DMSpectrum:
    spectab=['e','b','μ','γ','τ']

    def __init__(self,path,mDM,sigv,spec_in):
        self.path=path
        self.mDM=mDM
        self.sigv=sigv
        self.spec=spec_in
        self.dNdEtab,self.norm=self.build_spectrum_table()
        self.interp_func=self.create_interpolator()
        self.dNdE_func=self.make_dNdE()
    
    def build_spectrum_table(self):
        if (self.mDM <= 5) and (self.spec=='e'):
            logm=np.format_float_positional(np.log10(self.mDM),precision=3)
            spectra_name=os.path.join(self.path,'Spectra_e',f'{logm}.txt')
            norm_name=os.path.join(self.path,'Spcetra_e',f'norm_{logm}.txt')
            if not (os.path.exists(spectra_name) and os.path.exists(norm_name)):
                raise FileNotFoundError("Required spectral files not found.")
            dNdEtab=np.loadtxt(spectra_name)
            norm=float(np.loadtxt(norm_name))
        else:
            filename=os.path.join(self.path,'pppc','AtProduction_positrons','AtProduction_positrons.dat')
            if not os.path.exists(filename):
                raise FileNotFoundError("AtProduction_positrons.dat not found.")
            data=np.loadtxt(filename,skiprows=1)
            data=data[np.isclose(data[:,0],self.mDM)]
            if data.size == 0:
                raise ValueError("No data found for the specified DM mass.")
            if self.spec=='e':
                index=4
            elif self.spec=='b':
                index=13
            energies=data[:,1]
            x_vals=10**energies
            E_vals=x_vals*self.mDM
            dNdlogx=data[:,index]
            dNdE_vals=dNdlogx/(E_vals*np.log(10))
            dNdEtab=np.column_stack((E_vals,dNdE_vals))
            norm=1.0
        
        dNdEtab=np.vstack((dNdEtab,[1e10,0.0]))
        return dNdEtab,norm
    
    def create_interpolator(self):
        energies=self.dNdEtab[:,0]
        values =self.dNdEtab[:,1]
        return interp1d(energies,values,kind='linear',bounds_error=False,fill_value=0.0)
    
    def make_dNdE(self):
        def integrand(x):
            return self.interp_func(x)
        
        gaussian_norm,_=quad(integrand,0.7*self.mDM,self.mDM)
        gaussian_norm*=self.norm

        def dNdE(x):
            if not (1e-8*self.mDM<=x<=self.mDM):
                return 0.0
            if self.spec=='e':
                if x<=0.7*self.mDM:
                    return float(self.interp_func(x))
                sigma=0.2*self.mDM
                prefactor=gaussian_norm*2/np.sqrt(2*np.pi*sigma**2)
                return prefactor*np.exp(-(x-self.mDM)**2/(2*sigma**2))
            else:
                return float(self.interp_func(x))
        
        return dNdE
    
    def evaluate(self,x):
        return self.dNdE_func(x)
    
if __name__ == "__main__":
    path=Path.cwd()
    input_file=os.path.join(path,'input.txt')

    dm_spectrum=DMSpectrum(path,10,1e-28,'e')
    x_test=dm_spectrum.mDM
    print(f'dNdE({x_test})={dm_spectrum.evaluate(x_test)}')


### 6.3 Numerical Integration

In [ ]:
def save_fe(path, ii, jj, data):
    outfile = Path(path)/"f_e" /f"r_{ii}"/f"theta_{jj}.npy"
    os.makedirs(outfile.parent, exist_ok=True)
    np.save(outfile, data)

def calculatefe(mDM,sigv,spec):
    classify = RegionClassifier()
    parame=Parameters()
    phy = Physics()
    vel=Velocity()
    rho=BH_halo(a_J=0.9375,m_DM=mDM,sigv=sigv).rho
    path=str(Path.cwd())
    dm_spectrum=DMSpectrum(path,mDM,sigv,spec)

    path = Path.cwd()
    cutoff_data=np.load(path/"cutoffPoint.npy")
    p_inj_folder=os.path.join(path,"p_inj" if mDM>1 else "p_inj_subGeV")
    r_vals = np.unique(cutoff_data[...,0])
    theta_vals = np.unique(cutoff_data[...,1])
    p0_vals = np.unique(cutoff_data[...,2])
    nn = len(r_vals)-1
    mm = len(theta_vals)-1
    ll=len(p0_vals)

    total_points = (mm+1)*(nn+1)
    progress=tqdm(total=total_points, desc="Computing DM integral")
    for ii,r in enumerate(r_vals,start=1):
        for jj, theta in enumerate(theta_vals,start=1):
            region= classify.classify(r, theta)
            if region=='jet-disk':
                out=np.zeros((ll,2),dtype=float)
                out[:,0]=np.repeat(p0_vals,ll)
                save_fe(path, ii, jj, out)
                progress.update(1)
                continue
            if region=='jet':
                phi=phy.Phi(r,theta)
                thetaInj=phy.theta_inj_jet
                vR=vel.v_jet
                xi=xi_jet_interp
            else:
                thetaInj=phy.theta_stag
                vR=vel.v_disk
                xi=xi_disk_interp

            p_inj_folder=Path(p_inj_folder)
            target_path=p_inj_folder/ f"r_{ii}"/ f"theta_{jj}.npy"
            pinjtab= np.load(target_path)
            nrows=pinjtab.shape[0]
            nblocks=nrows//ll
            
            fe=np.zeros((ll,2),dtype=float)
            for vari in range(ll):
                p0=pinjtab[vari*nblocks,0]
                total=0.0
                for m in range(ll-1):
                    idx=vari*ll+m
                    r_inj=pinjtab[idx,1]
                    dr=abs(r_inj-pinjtab[idx+1,1])
                    p_inj=pinjtab[idx,2]
                    if region=='jet':
                        theta_inj=thetaInj(r_inj,phi)
                    else:
                        theta_inj=thetaInj(r,theta,r_inj)
                    vR_inj=vR(r_inj,theta_inj)

                    term=(parame.pc2cm/(parame.c*8*np.pi))*((sigv*rho(r_inj,theta_inj)**2)/(p_inj**2*mDM**2))*dm_spectrum.evaluate(p_inj)
                    term*=(1/vR_inj)*(p_inj/p0)**4*(xi(r,theta)/xi(r_inj,theta_inj))**4
                    term*=dr
                fe[vari]=(p0,total)
            save_fe(path, ii, jj, fe)
            progress.update(1)
    progress.close()
    return None

if __name__=="__main__":
    mDM=10
    sigv=1e-28
    spec='e'
    calculatefe(mDM,sigv,spec)

In [3]:
datafe=np.load("f_e/r_1/theta_2.npy")
for i in range(datafe.shape[0]):
    print(datafe[i,0],datafe[i,1])

0.001 1.7184064569139607e-06
10000.0 1.7139056239507751e-34
